<a href="https://colab.research.google.com/github/Shristy0412/Analysis-of-Epigenetic-Modifications/blob/main/Cancer_epigenetics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel('/content/GBM_METHYLATION_SLC8A1.xlsx')

In [3]:
print(data.head())

         RT         T       T.1       T.2       T.3       T.4      RT.1  \
0  0.929843  0.928156  0.921526  0.931479  0.889860  0.939439  0.948523   
1  0.464770  0.559134  0.460484  0.522254  0.815574  0.100259  0.523194   
2  0.922723  0.948616  0.943878  0.947867  0.899935  0.931495  0.941284   
3  0.509907  0.662649  0.505616       NaN       NaN       NaN  0.829613   
4  0.817826  0.832559  0.796367  0.882133  0.829623  0.918665  0.816905   

        T.5       T.6       T.7  ...     T.130     T.131     T.132     T.133  \
0  0.937603  0.917144  0.922838  ...  0.782110  0.955063  0.945038  0.954831   
1  0.760180  0.403001  0.606318  ...  0.286152  0.488535  0.603420  0.603572   
2  0.924937  0.934264  0.912804  ...  0.928383  0.941274  0.910866  0.942283   
3       NaN       NaN       NaN  ...  0.697143  0.480606  0.696004  0.689964   
4  0.913478  0.881346  0.854353  ...  0.892566  0.873461  0.873157  0.812158   

      T.134     T.135     T.136     T.137     T.138     T.139  
0  0

In [4]:
#Check for empty cells
if data.isnull().values.any():
    print("There are missing values in the dataset.")
else:
    print("No missing values detected.")

There are missing values in the dataset.


In [5]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)


In [6]:
# Assuming first row is labels, separate them
labels = data_imputed.iloc[0]
features = data_imputed.iloc[1:]

# Transpose for easier processing
features = features.T
features.columns = [f'CpG_{i+1}' for i in range(features.shape[1])]

# Map labels to 0, 1, 2 for normal, tumor, recurrent tumor
label_mapping = {'N': 0, 'T': 1, 'RT': 2}
labels_mapped = labels.map(label_mapping)

# Combine labels with features
features['label'] = labels_mapped.values


In [7]:
from sklearn.model_selection import train_test_split

X = features.drop('label', axis=1)
y = features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
if y_train.isna().sum() > 0:
    # Option 1: Remove rows with NaN values in y_train
    y_train = y_train.dropna()
    X_train = X_train.loc[y_train.index]

In [9]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_train shape: (0, 49)
y_train shape: (0,)


In [10]:
print("Initial DataFrame shape:", data_imputed.shape)
data_imputed_cleaned = data_imputed.dropna()  # Example cleaning step
print("Cleaned DataFrame shape:", data_imputed_cleaned.shape)


Initial DataFrame shape: (50, 155)
Cleaned DataFrame shape: (50, 155)


In [12]:
print(data_imputed_cleaned.columns)


Index(['RT', 'T', 'T.1', 'T.2', 'T.3', 'T.4', 'RT.1', 'T.5', 'T.6', 'T.7',
       ...
       'T.130', 'T.131', 'T.132', 'T.133', 'T.134', 'T.135', 'T.136', 'T.137',
       'T.138', 'T.139'],
      dtype='object', length=155)


In [14]:
# Example: Assuming 'N', 'RT', 'T' are in the first row of your dataset
data_imputed_cleaned['label'] = data_imputed_cleaned.iloc[0]  # or however your labels are defined


In [15]:
# Example: Assuming 'N', 'RT', 'T' are in the first row of your dataset
data_imputed_cleaned['label'] = data_imputed_cleaned.iloc[0]  # or however your labels are defined


In [16]:
print(data_imputed_cleaned.head())


         RT         T       T.1       T.2       T.3       T.4      RT.1  \
0  0.929843  0.928156  0.921526  0.931479  0.889860  0.939439  0.948523   
1  0.464770  0.559134  0.460484  0.522254  0.815574  0.100259  0.523194   
2  0.922723  0.948616  0.943878  0.947867  0.899935  0.931495  0.941284   
3  0.509907  0.662649  0.505616  0.486228  0.592723  0.527048  0.829613   
4  0.817826  0.832559  0.796367  0.882133  0.829623  0.918665  0.816905   

        T.5       T.6       T.7  ...     T.131     T.132     T.133     T.134  \
0  0.937603  0.917144  0.922838  ...  0.955063  0.945038  0.954831  0.901573   
1  0.760180  0.403001  0.606318  ...  0.488535  0.603420  0.603572  0.581026   
2  0.924937  0.934264  0.912804  ...  0.941274  0.910866  0.942283  0.920997   
3  0.782793  0.449169  0.753292  ...  0.480606  0.696004  0.689964  0.804824   
4  0.913478  0.881346  0.854353  ...  0.873461  0.873157  0.812158  0.854601   

      T.135     T.136     T.137     T.138     T.139  label  
0  0.91

In [17]:
X = data_imputed_cleaned.drop('label', axis=1)  # Replace 'label' with the correct column name
y = data_imputed_cleaned['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
# Check for NaN values in y_test
print(y_test.isnull().sum())

# Remove rows with NaN values in y_test and corresponding rows in X_test
idx = y_test[~y_test.isnull()].index
y_test = y_test.loc[idx]
X_test = X_test.loc[idx]

# Fit the model on the cleaned data
model.fit(X_test, y_test)

10


ValueError: Found array with 0 sample(s) (shape=(0, 155)) while a minimum of 1 is required by LogisticRegression.

In [22]:
# Check for NaN values in y_test
print(y_test.isnull().sum())

# Only remove rows with NaN values if there are any
if y_test.isnull().sum() > 0:
    # Remove rows with NaN values in y_test and corresponding rows in X_test
    idx = y_test[~y_test.isnull()].index
    y_test = y_test.loc[idx]
    X_test = X_test.loc[idx]

# Fit the model
# Check if X_test and y_test are not empty before fitting
if len(X_test) > 0 and len(y_test) > 0:
    model.fit(X_test, y_test)
else:
    print("Cannot fit the model. X_test or y_test is empty.")

0
Cannot fit the model. X_test or y_test is empty.
